In [12]:
import os
from dotenv import load_dotenv
import json
from instagram_network_capture import capturar_multiplas_paginas
import duckdb as db
import pandas as pd
from datetime import datetime

# Conexões do DuckDB com extensão JSON
con = db.connect()
con.install_extension('json')
con.load_extension('json')

# Define a data atual
hoje = datetime.now().strftime('%Y%m%d')

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

True

In [7]:
# Instala o python-dotenv se não estiver instalado
try:
    import dotenv
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])

In [8]:
def tratar_link_insta(link):
    user = str(link).split('/')[3]
    return user

In [9]:
df1 = pd.read_excel('Perfis testes - Novembro.xlsx', sheet_name='Hyeser')
df2 = pd.read_excel('Perfis testes - Novembro.xlsx', sheet_name='Fabio')
df2 = df2[['LINK', 'Rede']]
df2 = df2.query("Rede == 'Instagram'")
df = pd.concat([df1, df2], ignore_index=True)
df['LINK'] = df['LINK'].apply(lambda x: tratar_link_insta(x))
lista_usernames = list(df['LINK'])
max_users = int(len(lista_usernames))
max_users

59

In [13]:
capturar_multiplas_paginas(lista_usuarios=lista_usernames, usuario_login=str(os.getenv("LOGIN")), senha_login=str(os.getenv("SENHA")), delay=5)

Iniciando... [59 páginas]

Login...
✓ Login realizado

Capturando:
  ✓ smsindelicado
  ✓ espetacular
  ✓ motivei
  ✓ valordamente
  ✓ passandotempo
  ✓ obstinado.br
  ✓ megaincrivel
  ✓ acordeipravida
  ✓ despertei
  ✓ sensacional
  ✓ respostademulher
  ✓ resilienciamilionaria
  ✓ semspoiler
  ✓ vencinavida
  ✓ palavrasboas
  ✓ tribovisionaria
  ✓ reflitars
  ✓ deusvaitesurpreender
  ✓ mentesapiente
  ✓ bemironico
  ✓ deuspodemudartudo
  ✓ musicaboa
  ✓ vozdoevangellhoo
  ✓ apnas.existindo
  ✓ tip0grafia2000
  ✓ deus24hr
  ✓ traumatizaram
  ✓ bilhetesdiariamente_
  ✗ deusfaloueucrerio - Endpoint não encontrado
  ✓ linguagem.secreta
  ✓ conectadoscomjesus_
  ✓ ig.apaixonados
  ✓ vivapradeus
  ✓ receitas_veganas_simples_
  ✓ churrasco_dicas_e_receitas
  ✓ expressandofe
  ✓ dediqueiumverso
  ✓ sobreautoestima
  ✓ bilhetessincero
  ✓ receitas.criativass
  ✓ citouemfrases
  ✓ cachorros_sincero
  ✓ uma.dose.de.caos
  ✓ sigofraseando
  ✓ aquella_musica
  ✓ paradamusicais
  ✓ deusfalandocomvoc

In [60]:
for item in os.listdir('teste_json'):
    if item.endswith('.json'):
        with open(os.path.join('teste_json', item), 'r', encoding='utf-8') as f:
            teste = json.load(f)
            novo = teste['require'][0][3][0]['__bbox']['require'][0][3][1]['__bbox']['result']['data']['xdt_api__v1__feed__reels_media']['reels_media']
        with open(os.path.join('teste_json', item), 'w', encoding='utf-8') as f:
            json.dump(novo, f, ensure_ascii=False, indent=4)

In [85]:
path = r'teste_json\*.json'
df = con.execute('''
with pt1 as (
SELECT
user.username AS username,
unnest(items) AS item
FROM read_json_auto(?, ignore_errors=true)
),
pt2 as (
SELECT
username,
replace(split(split(unnest(item.story_link_stickers).story_link.url, 'u=')[2], '%2F')[3], 'www.', '') AS story_link_url
FROM pt1
)
select
username,
story_link_url,
case
when story_link_url = 'amzlink.to' then 'Amazon'
when story_link_url = 'mercadolivre.com' then 'Mercado Livre'
when story_link_url = 's.shopee.com.br' then 'Shopee'
when story_link_url = 'minhaloja.natura.com' then 'Natura'
when story_link_url = 'magazinevoce.com.br' then 'Magazine Luiza'
when story_link_url = 'elausa.com.br' then 'Ela Usa'
when story_link_url = 'epocacosmeticos.com.br' then 'Época Cosméticos'
when story_link_url = 'natura.com.br' then 'Natura'
when story_link_url = 'sminhaloja.natura.com' then 'Natura'
when story_link_url = 'api.whatsapp.com' then 'WhatsApp'
when story_link_url = 'google.com' then 'Google'
when story_link_url = 'encurtador.com.br' then 'Encurtador'
else null end as origin
from pt2
''', [path]).df()
df.to_csv(f'output_final_{hoje}.csv', index=False)

In [86]:
df_val = pd.read_csv(f'output_final_{hoje}.csv')
df_val.username.nunique()

51